

# **XGBRegressor and LGBMRegressor**











## **Resources**

- XGBRegressor
https://machinelearningmastery.com/xgboost-for-regression/

- LGBMRegressor
https://www.datatechnotes.com/2022/03/lightgbm-regression-example-in-python.html


## **Reading the Data**

### Importing Needed packages


In [12]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
import os
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.utils import shuffle


### Reading the Data


In [17]:
current_directory = os.getcwd() 
parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
print("Parent directory:", parent_directory)
# Data_directory = parent_directory + "/Data" + "/FuelConsumptionCo2.csv"
Data_directory = parent_directory + "/Data" + "/dataset_reduce_PI_MS.csv"
print(Data_directory)

Parent directory: d:\Github\Resources
d:\Github\Resources/Data/dataset_reduce_PI_MS.csv


In [18]:
df = pd.read_csv(Data_directory)
# cdf = df[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_CITY','FUELCONSUMPTION_HWY','FUELCONSUMPTION_COMB','CO2EMISSIONS']]
cdf = df[['tau','theta','MVI','Deli', 'AcorAr','MG']]
cdf.head(9)

,tau,theta,MVI,Deli,AcorAr,MG
0,455,0.999889,0.000111,0.000046,354.950224,3.166541
1,55,0.999773,0.000222,0.000124,471.653984,4.015505
2,105,0.999969,0.000024,0.000024,7227.140088,8.744351
3,255,0.999980,0.000019,0.000014,8618.695738,6.210367
4,55,0.999933,0.000046,0.000059,2122.803986,5.213637
5,405,0.999733,0.000243,0.000130,52.425919,3.769319
6,105,0.999982,0.000012,0.000007,13166.795610,2.885962
7,155,0.999978,0.000020,0.000015,11614.021570,6.810561
8,5,0.978437,0.021558,0.002707,9.249500,3.101440


#### Creating train and test dataset



In [19]:
msk = np.random.rand(len(df)) < 0.8
train = cdf[msk]
test = cdf[~msk]


col= ['tau','theta','MVI','Deli', 'AcorAr']    
X_train = np.asanyarray(train[col])
y_train = np.asanyarray(train[['MG']])
X_test= np.asanyarray(test[col])
y_test = np.asanyarray(test[['MG']])
X=np.concatenate([X_test, X_train], axis=0)
y=np.concatenate([y_test, y_train], axis=0)

## **XGBoost**


In [20]:
import xgboost 
from xgboost import XGBRegressor

In [21]:
# create an xgboost regression model
model = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

- **n_estimators**: The number of trees in the ensemble, often increased until no further improvements are seen.
- **max_depth**: The maximum depth of each tree, often values are between 1 and 10.
- **eta**: The learning rate used to weight each model, often set to small values such as 0.3, 0.1, 0.01, or smaller.
- **subsample**: The number of samples (rows) used in each tree, set to a value between 0 and 1, often 1.0 to use all samples.
- **colsample_bytree**: Number of features (columns) used in each tree, set to a value between 0 and 1, often 1.0 to use all features.

In [26]:
    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import RepeatedKFold
    from numpy import absolute

    # create an xgboost regression model
    model = XGBRegressor(n_estimators=1000, max_depth=10, eta=0.01, subsample=0.7, colsample_bytree=1)
    # define model evaluation method
    cv = RepeatedKFold(n_splits=20, n_repeats=2, random_state=1)
    # evaluate model
    scores = cross_val_score(model, X, y, scoring='r2', cv=cv, n_jobs=-1)
    # force scores to be positive
    scores = absolute(scores)
    print('Mean R2: %.3f (%.3f)' % (scores.mean(), scores.std()))

Mean R2: 0.987 (0.017)


In [24]:
# create an xgboost regression model
model = XGBRegressor(n_estimators=1000, max_depth=10, eta=0.01, subsample=0.7, colsample_bytree=1)
model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False, eta=0.01,
             eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)

In [25]:
from sklearn.metrics import r2_score
# Make predictions on the test set
test_predictions = model.predict(X_test)
# Calculate R2 on the test predictions
r2 = r2_score(y_test, test_predictions)
print('R2 on test set:', r2)

R2 on test set: 0.9853583885458287


## **LGBMRegressor**

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from numpy import absolute

# Import LGBMRegressor
from lightgbm import LGBMRegressor

# create a LightGBM regression model
model = LGBMRegressor(n_estimators=1000, max_depth=10, learning_rate=0.01, subsample=0.7, colsample_bytree=1)
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=4, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='r2', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean R2: %.3f (%.3f)' % (scores.mean(), scores.std()))

Mean R2: 0.960 (0.019)
